In [ ]:
#Importing all the libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset,DataLoader
from torchvision import models,datasets,transforms

from tqdm import tqdm
import os
from PIL import Image
import matplotlib.pyplot as plt
import math
import random
import pickle

In [ ]:
#Checking if a GPU with CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
#Creating a custom dataset class that generates the noisy counterpart of the clean image and returns both of them
class CustomDataset(Dataset):
  def __init__(self,dir_name,train_flag,sigma):
    """
    train_flag is True for the Train dataset and False for the evaluation dataset
    sigma is the noise level
    """
    super().__init__()
    #Defining the transforms for the train and test datasets
    self.test_transform = transforms.Compose([transforms.ToTensor()])

    self.train_flag = train_flag

    #Downloading the train and test datasets
    if train_flag:
      pfile = open(dir_name, 'rb')     #opening the pickle file
      self.dataset = pickle.load(pfile) #Reading the tensors from the pickle file
      pfile.close() #Closing the pickle file

    else:
      self.dataset = []
      for img_name in os.listdir(dir_name):
        self.dataset.append(os.path.join(dir_name,img_name))

    self.sigma = sigma

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, index):
    if self.train_flag:
      clean_img = self.dataset[index].unsqueeze(0) #of shape (1,40,40)

    else:
      im = Image.open(self.dataset[index])
      clean_img = self.test_transform(im) #of shape (1,h,w)

    #Generating the noisy image
    noisy_img = (torch.randn(clean_img.shape)*(self.sigma/255.))+clean_img
    return clean_img, noisy_img



In [ ]:
#Defining a Block of the DnCNN
class Block(nn.Module):
  def __init__(self,k=3,p=1,c=64):
    super().__init__()
    self.conv = nn.Conv2d(in_channels=c,out_channels=c,kernel_size=k,padding=p,bias=False) #same padding convolution
    self.norm = nn.BatchNorm2d(c) #batch normalization
    self.relu = nn.ReLU(inplace=True) #activation function

  def forward(self,x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.relu(x)
    return x

In [ ]:
#Defining the DnCNN model
class DCNN(nn.Module):
  def __init__(self,k=3,p=1,c=64,l=17,in_c=1):
    super().__init__()
    layers = [nn.Conv2d(in_channels=in_c,out_channels=c,kernel_size=k,padding=p,bias=False), #First same padding convolution layer
              nn.ReLU(inplace=True)]
    layers.extend([Block(k,p,c) for _ in range(l-2)]) #Adding all the "Blocks" to the model
    layers.append(nn.Conv2d(in_channels=c,out_channels=in_c,kernel_size=k,padding=p,bias=False)) #Last same padding convolution layer

    self.all = nn.Sequential(*layers)

  def forward(self,x):
    out = self.all(x)
    return x-out


In [ ]:
#Function for supervised training of the model
def train(criterion,optimizer,model,device,train_loader):
  """
  criterion is the loss function
  optimizer is the optimization algorithm used
  model is the denoiser model
  device is either CPU or GPU(cuda)
  train_loader is the DataLoader containing the training dataset
  """

  model.train()
  loop = tqdm(train_loader)  #Used to visualized the progress in training
  cur_loss = 0.0

  for i,(clean,noisy) in enumerate(loop):   #iterating batch-by-batch through the dataset
    clean,noisy = clean.to(device), noisy.to(device) #Moving over the data to the "device"
    clean_pred = model(noisy) #Passing the data through the model
    loss = criterion(clean_pred,clean) #Computing the loss

    optimizer.zero_grad() #Zeroing all the previous gradients
    loss.backward() #Computing the gradients for the current iteration
    optimizer.step() #Updating the weights of the model

    cur_loss += loss.item() #Keeping track of the loss
    loop.set_postfix(loss=cur_loss/(i+1)) #Printing the cumulative loss after each iteration

In [ ]:
#Function for testing the model
def test(criterion,model,device,test_loader):
  """
  criterion is the function used to compute PSNR
  model is the denoiser model
  device is either CPU or GPU(cuda)
  test_loader is the DataLoader containing the test dataset
  """

  model.eval()
  loop = tqdm(test_loader) #Used to visualized the progress in testing
  total_mse = []

  with torch.no_grad(): #Ensures that the gradients are not computed
    for i,(clean,noisy) in enumerate(loop): #iterating batch-by-batch through the dataset
      clean,noisy = clean.to(device).float(), noisy.to(device).float() #Moving over the data to the "device"
      out = model(noisy) #Passing the data through the model

      clean_pred = torch.clamp(out,min=0.0,max=1.0) #Clips all the values greater than 1 or less than 0
      loss = (criterion(clean_pred,clean).mean(axis=(1,2,3))).tolist() #Computing MSE at an image level
      total_mse.extend(loss)

  total_mse_tensor = torch.tensor(total_mse)
  psnr = (-10*torch.log10(total_mse_tensor)).mean() #Computing the PSNR using the corresponding MSE values

  print(f"The PSNR is {psnr}")
  return psnr.item()

In [ ]:
#Function to computer the number of parameters in a model
def number_of_parameters(model):
    return sum(params.numel() for params in model.parameters() if params.requires_grad)

In [ ]:
#Wrapper function to train and evaluate the denoiser model
def wrapper(sigma):
  print(f"This is for sigma of {sigma}")

  #Defines the loaders for the train and test set
  train_set = CustomDataset("/content/drive/MyDrive/R2R_ML/BSD400_train.pkl",True,sigma)
  test_set = CustomDataset("/content/drive/MyDrive/R2R_ML/test",False,sigma)

  train_loader = DataLoader(train_set,batch_size=128,shuffle=True,num_workers=128)
  test_loader = DataLoader(test_set,batch_size=1,shuffle=False,num_workers=1)

  print(f"The number of images in the train set is {len(train_set)}")
  print(f"The number of images in the test set is {(len(test_set))}")

  #Defining the model, loss function and optimizer
  model = DCNN().to(device)
  criterion_train = nn.MSELoss()
  criterion_test = nn.MSELoss(reduce=False)
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  epochs = 20

  print(f"The model has {number_of_parameters(model)} parameters")
  #Computing the PSNR between the noisy and clean image
  total_mse = []
  with torch.no_grad():
      for i,(clean,noisy) in enumerate(test_loader):
        clean,noisy = clean.to(device), noisy.to(device)
        loss = (criterion_test(noisy,clean).mean(axis=(1,2,3))).tolist()
        total_mse.extend(loss)

  total_mse_tensor = torch.tensor(total_mse)
  psnr = (-10*torch.log10(total_mse_tensor)).mean()
  print(f"The PSNR for an untrained densoiser is {psnr}")

  #Iterating through the epochs
  for epoch in range(epochs):
    print(f"The current epoch is {epoch}")
    train(criterion_train,optimizer,model,device,train_loader)
    cur_psnr = test(criterion_test,model,device,test_loader)
    torch.save(model.state_dict(), "Supervised"+str(epoch)+"_"+str(round(cur_psnr,2))+"_"+ str(sigma) + ".pt")


In [ ]:
#For sigma value of 10
wrapper(10)

This is for sigma of 10


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 128 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The number of images in the train set is 238400
The number of images in the test set is 68
The model has 556032 parameters


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


The PSNR for an untrained densoiser is 28.131332397460938
The current epoch is 0


...

The PSNR is 33.79368591308594
The current epoch is 18


100%|██████████| 68/68 [00:01<00:00, 47.24it/s]


The PSNR is 33.79641342163086
The current epoch is 19


100%|██████████| 68/68 [00:01<00:00, 47.58it/s]


The PSNR is 33.79227828979492


In [ ]:
#For sigma value of 25
wrapper(25)

This is for sigma of 25
The number of images in the train set is 238400
The number of images in the test set is 68
The model has 556032 parameters
The PSNR for an untrained densoiser is 20.168657302856445
The current epoch is 0


100%|██████████| 68/68 [00:01<00:00, 47.81it/s]


The PSNR is 28.13762092590332
The current epoch is 1


100%|██████████| 68/68 [00:01<00:00, 48.31it/s]


The PSNR is 28.514789581298828
The current epoch is 2


...

The PSNR is 23.36663818359375
The current epoch is 18


100%|██████████| 68/68 [00:01<00:00, 46.16it/s]


The PSNR is 28.993730545043945
The current epoch is 19


100%|██████████| 68/68 [00:01<00:00, 46.35it/s]


The PSNR is 29.080610275268555


In [ ]:
#For sigma value of 50
wrapper(50)

This is for sigma of 50
The number of images in the train set is 238400
The number of images in the test set is 68
The model has 556032 parameters
The PSNR for an untrained densoiser is 14.154053688049316
The current epoch is 0


100%|██████████| 68/68 [00:01<00:00, 43.04it/s]


The PSNR is 24.96471405029297
The current epoch is 1


100%|██████████| 68/68 [00:01<00:00, 46.24it/s]


The PSNR is 25.299646377563477
The current epoch is 2


...

The PSNR is 26.020998001098633
The current epoch is 18


100%|██████████| 68/68 [00:01<00:00, 37.89it/s]


The PSNR is 25.144298553466797
The current epoch is 19


100%|██████████| 68/68 [00:01<00:00, 45.75it/s]


The PSNR is 26.09442138671875
